## The model
***
In this project we use a Aspergilius niger GSM-model developed by DTU (CBS513.88) and modify it so it is able to produce recombinant THC. To be able to do this we need to modify the organism by adding metabolites and reaction. 

In [1]:
# Importing libraries
from cobra.io import read_sbml_model
from cobra import Metabolite, Reaction
import cobra

# Define the model
model = read_sbml_model('GSM_A.niger.xml')

'' is not a valid SBML 'SId'.


## Metabolites, Reactions and Enzymes to be added in model
In order to make recombinant THC several enzymes and metabolites need to be added in the model, as Aspergilius 
***
Metabolites

- Hexanoyl-coA (HcoA) - C27H46N7O17P3S - in model
- Olivetolic Acid (OA) - C12H16O4 - add
- (IPP) Isopentenyl pyrophosphate (IPP, isopentenyl diphosphate, or IDP) - 	C5H12O7P2 - in model
- Dimethylallyl pyrophosphate (DMAPP) -  C5H12O7P2 - add
- Geranyl diphosphate (GPP) - C10H20O7P2 - in model 
- Cannabigerolic acid (CBGA) - C22H32O4 - add
- Tetrahydrocannabinolic acid (THCA) - C22H30O4 -add
- Malonyl_CoA  - in model

Reactions 

- Hexanoyl-coA + 3 malonyl-CoA -> Olivetoic acid (OA)                                                    (1) OLS
- Dimethylallyl diphosphate (DMAPP) + IPP -> Geranyl diphosphate (GPP)       (2) GPPS
- Geranyldiphosphate (GPP) + Olivetolic acid (OA) -> cannabigerolic acid (CBGA)                          (3) CBGAS
- cannabigerolic acid (CBGA) -> Tetrahydrocannabinolic acid (THCA)                                       (4) THCAS
- Tetrahydrocannabinolic acid (THCA) ->                                             (5) Export
-  IPP -> DMPP                                                                       (6) evenwicht

Enzymes
- Olivetol synthase (OLS)        Gene:   OC115699293
- Olivetolic acid cyclase (OAC)  Gene:   OC115723438 
- CBGA synthase (CBGAS)          Gene:   PT4-T
- THCA synthase (THCAS) 	     Gene:   SAPIO_CDS2003
- GPP synthase (GPPS)            Gene:   ERG20 (S. cerevisiea)
- Isopentenyl-diphosphate isomerase Gene: fcd1 (Solanum lycopersicum)
***

In [2]:
# Get id vof metabolites already present in the model
hexanoyl_coA =model.metabolites.get_by_id('C60COA')
malonyl_CoA = model.metabolites.get_by_id('MALCOA')
isopentenyl = model.metabolites.get_by_id('IPPP')
geranyl = model.metabolites.get_by_id('GPP')

# Create metabolites that need to be added to the model
olivetolic_acid = cobra.core.Metabolite(id = 'OA', formula = 'C12H16O4', name = 'Olivetolic Acid', charge = 0, compartment = 'c')
dmapp = cobra.core.Metabolite(id = 'DMAPP', formula = 'C5H12O7P2', name = 'Dimethylallyl diphosphate', charge = 0, compartment = 'c')
cannabigerolic_acid= cobra.core.Metabolite(id = 'CBGA', formula = 'C22H32O4', name = 'Cannabigerolic Acid', charge = 0, compartment = 'c')
tetrahydrocannabinolic_acid  = cobra.core.Metabolite(id = 'THCA', formula = 'C22H30O4', name = 'Tetrahydrocannabinolic acidt', charge = 0, compartment ='c')

# Addition of metabolites to the model
model.add_metabolites(olivetolic_acid)
model.add_metabolites(dmapp)
model.add_metabolites(cannabigerolic_acid)
model.add_metabolites(tetrahydrocannabinolic_acid)


In [3]:
# Creation missing reactions of TCH biosynthetic pathway

dmapps = cobra.core.Reaction(id = 'DMAPPS', name = 'Isopentenyl-diphosphate isomerase' , lower_bound = 0 , upper_bound = 1000 )
dmapps.add_metabolites({isopentenyl : -1, dmapp:1}, reversibly = False)
dmapps.gene_reaction_rule = '(fcd1)'

gpps = cobra.core.Reaction(id = 'GPPS', name = 'Geranyl-diphosphate synthase' , lower_bound = 0 , upper_bound = 1000 )
gpps.add_metabolites({dmapp : -1, isopentenyl:-1, geranyl: 1}, reversibly = False)
gpps.gene_reaction_rule = '(ERG20)'

ols = cobra.core.Reaction(id = 'OLS', name = 'Olivetol Syntase' ,   lower_bound = 0 , upper_bound = 1000 )
ols.add_metabolites({ hexanoyl_coA : -1 , malonyl_CoA : -3, olivetolic_acid: 1}, reversibly = False)
ols.gene_reaction_rule = '(OC115699293)'

cbgas = cobra.core.Reaction(id = 'CBGAS', name = 'Cannabigerolic acid synthase' , lower_bound = 0 , upper_bound = 1000 )
cbgas.add_metabolites({geranyl : -1, olivetolic_acid : -1, cannabigerolic_acid: 1 }, reversibly = False)
cbgas.gene_reaction_rule = '(PT4-T)'

thcas = cobra.core.Reaction(id = 'THCAS', name = 'Tetrahydrocannabinolic acid synthase' ,  lower_bound = 0 , upper_bound = 1000 )
thcas.add_metabolites({cannabigerolic_acid : -1, tetrahydrocannabinolic_acid: 1}, reversibly = False)
thcas.gene_reaction_rule = '(SAPIO_CDS2003)'

thca_e = cobra.core.Reaction(id = 'THCA_e', name = 'THCA export' , lower_bound = 0 , upper_bound = 1000 )
thca_e.add_metabolites({tetrahydrocannabinolic_acid : -1}, reversibly = False)

# Addition of reactions to the model
model.add_reactions((dmapps, gpps, ols, cbgas, thcas, thca_e))


***
### Setting the objective to THC production
***
Since the goal is to optimize THC production the objective has to be set to the "THCAS"  reaction 

In [5]:
model.objective = {model.reactions.THCAS: 1}
solution = model.optimize()
print('The maximum production rate of THC using the heterologous organism is {:.2f} mmol/h*gDw'.format(solution.fluxes.THCA_e))


The maximum production rate of THC using the heterologous organism is 3.56 mmol/h*gDw
